In [ ]:
import random

lyric_gen_data = pd.read_csv("preprocessed_lyrics.csv")

# Iterate over each unique artist in the dataset
for artist_name in lyric_gen_data["Artist"].unique():

    # Get the lyrics for the current artist
    artist_lyrics = lyric_gen_data.loc[lyric_gen_data["Artist"] == artist_name, "Lyrics"].tolist()
    tokenized_lyrics = [lyric.split() for lyric in artist_lyrics]

    # Create a dictionary of word tuples and following words
    word_dict = {}
    for lyric in tokenized_lyrics:
        for i in range(len(lyric) - 2):
            word_tuple = tuple(lyric[i:i+2])
            following_word = lyric[i+2]
            if word_tuple not in word_dict:
                word_dict[word_tuple] = []
            word_dict[word_tuple].append(following_word)

    # Create a list of starting tuples
    starting_tuples = []
    for lyric in tokenized_lyrics:
        if len(lyric) >= 2:
            starting_tuples.append((lyric[0], lyric[1]))

    # Set the maximum number of lines and line length for the song
    max_lines = 20
    max_line_length = 10

    # Create a folder for the generated lyrics for the specific artist
    generated_lyrics_dir = "generated Lyrics"
    artist_dir = os.path.join(generated_lyrics_dir, artist_name)
    if not os.path.exists(artist_dir):
        os.mkdir(artist_dir)

    # Select a random starting tuple from the word dictionary
    current_tuple = random.choice(starting_tuples)

    # Generate multiple lines of lyrics
    song_lines = []
    for i in range(max_lines):
        # Generate a new line of lyrics
        new_line = list(current_tuple)
        while len(new_line) < max_line_length:
            if current_tuple in word_dict:
                following_words = word_dict[current_tuple]
                next_word = random.choice(following_words)
                new_line.append(next_word)
                current_tuple = (current_tuple[1], next_word)
            else:
                break

        # Convert the new line to a string
        new_line_str = " ".join(new_line)

        # Check if the new line has already been generated
        if new_line_str in song_lines:
            break

        # Append the new line to the song and the list of generated lines
        song_lines.append(new_line_str)

    # Combine the song lines into a single song string
    song = "\n".join(song_lines)

    # Write the generated song to a file
    filename = os.path.join(artist_dir, "generated_song.txt")
    with open(filename, "w") as f:
        f.write(song)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import load_model


# Loop through the generated lyrics and predict the artist for each song
for artist_name in os.listdir("generated lyrics"):
    artist_dir = os.path.join("generated lyrics", artist_name)
    if os.path.isdir(artist_dir):
        all_lyrics = ''  # create an empty string to store all lyrics for an artist
        for filename in os.listdir(artist_dir):
            file_path = os.path.join(artist_dir, filename)
            with open(file_path, "r") as f:
                generated_lyrics = f.read().replace('\n', '') # remove new line character
                all_lyrics += generated_lyrics
                lyric_sequences = tokenizer.texts_to_sequences(all_lyrics)
                lyrics = pad_sequences(lyric_sequences, maxlen=200)
                y_pred_prob = model_mlp.predict(X_test)
                predictions = np.argmax(y_pred_prob, axis=1)

pre_processed_data2 = pd.read_csv("preprocessed_lyrics.csv")
artists2 = pre_processed_data2["Artist"].unique()
print(artists2)
label_encoder2 = LabelEncoder()
# pre_processed_data2["Artist"] = label_encoder2.fit_transform(pre_processed_data2["Artist"])
# print(pre_processed_data2["Artist"])
# encoded_artist = 2
# artist_name = label_encoder.inverse_transform([encoded_artist])[0]
# print(artist_name)


#Future thoughts, I could set up the generated songs and lyrics in a similar fashion to the classification model. So it would be in a 
# csv format, properly processed, and it would allow for easier predictions. I would have it produce 10 songs for each artist.